## ANN MLFlow Implementation

### Compare runs, choose a model and deploy it to a REST API
- Run a hyperparameter sweep on the training script
- Compare the results of the runs in the MLFLow UI
- Choose the best run and register it as a model
- Deploy the final model a REST API
- Build a container image suitable for deployment on cloud

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [2]:
# load the dataset from the url
# Get the path to the dataset relative to the notebook location
dataset_path = os.path.join(os.pardir, 'dataset', 'wine-quality.csv')
data = pd.read_csv(
    dataset_path,
    sep=";",
    )
data.head()[:5]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
# Split into train, test and validation sets
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)

# Define the input features
train_x = train_data.drop(columns=['quality']).values
train_y = train_data['quality'].values

test_x = test_data.drop(columns=['quality']).values
test_y = test_data['quality'].values

# Create val set from train set
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)


In [4]:
signature = infer_signature(train_x, train_y)

In [5]:
def train_model(
        params,
        epochs,
        train_x,
        train_y,
        val_x,
        val_y,
        test_x,
        test_y,
        ):
    
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1),
        ]
    )

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=params["lr"],
        momentum=params["momentum"]),
        loss = keras.losses.MeanSquaredError(),
        metrics = [keras.metrics.RootMeanSquaredError()],
        )

    # Train the model with lr and momentum parameters with ML Flow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(val_x, val_y),
            epochs=epochs,
            batch_size=32
            )
        
        # Evaluate the model
        eval_result = model.evaluate(test_x, test_y)

        eval_rmse = eval_result[1]

        # Log the parameters and metrics
        mlflow.log_params(params)
        mlflow.log_metric("rmse", eval_rmse)
        
        # Log the model
        mlflow.tensorflow.log_model(model, "ANN model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}
        

In [6]:
def objective(params):
    result = train_model(
        params=params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        val_x=val_x,
        val_y=val_y,
        test_x=test_x,
        test_y=test_y,
        )
    
    return result

In [7]:
space={
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(0.1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

### Training + tracking script below

In [8]:
# mlflow.set_tracking_uri(os.path.join(os.pardir, "mlruns"))
# mlflow.set_experiment("/wine-quality-ann")
# with mlflow.start_run():
#     trials=Trials()
#     best = fmin(
#         fn=objective,
#         space=space,
#         algo=tpe.suggest,
#         max_evals=4,
#         trials=trials
#     )

#     # Fetch the details of the best run
#     best_run = sorted(trials.results, key=lambda x: x['loss'])[0]

#     # log the best parameters, loss and model
#     mlflow.log_params(best)
#     mlflow.log_metric("eval_rmse", best_run['loss'])
#     mlflow.tensorflow.log_model(
#         best_run['model'],
#         "ANN model",
#         signature=signature,
#         )
    
#     print(f"Best run: {best_run}")
#     print(f"Best parameters: {best}")
#     print(f"Best loss: {best_run['loss']}")
        

### Inferencing

In [9]:
import mlflow

# Set the tracking URI to point to your mlruns directory
mlflow.set_tracking_uri(os.path.join(os.pardir, "mlruns"))

model_uri = 'models:/finalmodel/2'

# Replace INPUT_EXAMPLE with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
input_data = test_x

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="virtualenv",
)

2025/04/07 14:51:34 INFO mlflow.models.python_api: It is highly recommended to use `uv` as the environment manager for predicting with MLflow models as its performance is significantly better than other environment managers. Run `pip install uv` to install uv. See https://docs.astral.sh/uv/getting-started/installation for other installation methods.
/Users/saksham/Desktop/projects/mlflow/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025/04/07 14:51:34 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


MlflowException: Could not find the virtualenv binary. Run `pip install virtualenv` to install virtualenv.